# Predictions for Spotify

## Extraemos la data

In [54]:
import pandas as pd

In [55]:
# Read in the data
userinput_df = pd.read_csv('data/input.csv', index_col=0)
candidatos_df = pd.read_csv('data/datadmc.csv', index_col=0)

In [56]:
print(userinput_df.shape)
userinput_df.head()

(15, 13)


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
La Chata,3jzpdDnLaQcsuGCw4JwVYS,0.251,0.569,206840,0.525,0.000000,2,0.3460,-5.809,1,0.0257,153.988,0.303
Decir adiós,7c6Zu7NeLHWy4r3ztZuT0G,0.680,0.566,239000,0.360,0.000000,7,0.0972,-10.258,1,0.0315,134.124,0.431
Te Quiero,0eJGouVmEGxPktWPSTfpuI,0.463,0.454,264880,0.670,0.000002,0,0.0735,-6.704,1,0.0815,190.113,0.378
Decir Adiós,1snT64YtuTvZYVzuornRjw,0.463,0.596,232227,0.454,0.000001,7,0.1140,-7.032,1,0.0421,135.671,0.409
Fin del Tiempo,5xj5qixWeH2I44SFx0Icdv,0.692,0.561,186500,0.475,0.000000,9,0.2410,-5.918,1,0.0289,141.451,0.384


In [57]:
print(candidatos_df.shape)
candidatos_df.head()

(555, 13)


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
Unnamed: 0,,,,,,,,,,,,,
Estoy a la Puerta y Llamo,4bjsp9L4sWREYUkZU8nc6e,0.935,0.493,198240,0.1120,0.000558,10,0.1630,-14.419,0,0.0345,76.480,0.243
Nada Te Turbe,7GPBx7jwkMSnlUS6Qqudlp,0.859,0.523,216054,0.2420,0.000072,0,0.1050,-9.303,1,0.0269,103.928,0.319
Entraré,1koXbetRP1PgmDRqU4DFPb,0.944,0.426,262373,0.0559,0.000000,7,0.0633,-18.357,1,0.0300,71.843,0.116
Someone You Loved,7qEHsqek33rTcFNT9PFqLf,0.751,0.501,182161,0.4050,0.000000,1,0.1050,-5.679,1,0.0319,109.891,0.446
Before You Go,2gMXnyrvIjhVBUZwvLZDMP,0.604,0.459,215107,0.5750,0.000000,3,0.0885,-4.858,1,0.0573,111.881,0.183


## Sistema de recomendación

![el sistema de recomendación](./sistema_recomendacion.png)

Veamos en detalle el sistema de recomendación:

![el sistema de recomendación en detalle](./sistema_recomendacion_detalle.png)


#### El filtrado basado en contenido

Permite cuantificar qué tan similar es un ítem de `candidatos_df` a un ítem de `top20_df`.

Una forma de hacer esta comparación es usando la similitud del coseno:

![vectores de características](./vectores_caracteristicas.png)

![la similitud del coseno](./similitud_coseno.png)

Calcularemos este similitud entre cada user_input y cada una de las pistas candidatas (matriz de 20 x n_pistas_candidatas)

In [58]:
# Extraer sólo los features en formato numpy array
userinput_mtx = userinput_df.iloc[:,1:].values  # "1:" solo toma variables numéricas
candidatos_mtx = candidatos_df.iloc[:,1:].values

In [59]:
candidatos_mtx

array([[9.35000e-01, 4.93000e-01, 1.98240e+05, ..., 3.45000e-02,
        7.64800e+01, 2.43000e-01],
       [8.59000e-01, 5.23000e-01, 2.16054e+05, ..., 2.69000e-02,
        1.03928e+02, 3.19000e-01],
       [9.44000e-01, 4.26000e-01, 2.62373e+05, ..., 3.00000e-02,
        7.18430e+01, 1.16000e-01],
       ...,
       [1.12000e-01, 6.53000e-01, 1.60191e+05, ..., 5.02000e-02,
        8.39700e+01, 5.53000e-01],
       [1.80000e-01, 4.52000e-01, 2.19955e+05, ..., 4.58000e-02,
        1.41692e+02, 3.15000e-01],
       [3.01000e-01, 7.02000e-01, 2.07455e+05, ..., 1.30000e-01,
        7.96400e+01, 8.43000e-01]])

In [60]:
from sklearn.preprocessing import StandardScaler

# Estandarizar cada columna de features: mu = 0, sigma = 1
# pues cada característica tiene una escala diferente
scaler = StandardScaler()
user_scaled = scaler.fit_transform(userinput_mtx)
cand_scaled = scaler.fit_transform(candidatos_mtx)

In [61]:
print(user_scaled.mean(axis=0))
print(user_scaled.std(axis=0))

[-5.18104078e-17  2.07241631e-16 -2.51650552e-16 -1.33226763e-16
 -7.30896825e-17  2.96059473e-17  1.85037171e-16  6.21724894e-16
  1.03620816e-16 -5.08852220e-17  7.99360578e-16 -8.88178420e-17]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [62]:
import numpy as np

# Normalizar cada vector de características (es decir por filas)

# Magnitudes de cada vector (o pista)
user_norm = np.sqrt((user_scaled*user_scaled).sum(axis=1))
cand_norm = np.sqrt((cand_scaled*cand_scaled).sum(axis=1))

# Normalización
nuser = user_scaled.shape[0]
ncand = cand_scaled.shape[0]
user = user_scaled/user_norm.reshape(nuser,1)
cand = cand_scaled/cand_norm.reshape(ncand,1)

print(np.sqrt((user*user).sum(axis=1)))
print(np.sqrt((cand*cand).sum(axis=1)))


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

In [63]:
from sklearn.metrics.pairwise import linear_kernel

# Calcular similitudes del coseno entre cada top-20 y cada
# una de las pistas candidatas
cos_sim = linear_kernel(user,cand)
cos_sim.shape

(15, 555)

In [64]:
# Ejemplo: ¿qué tanto se parece una pista  user-input a una candidata?
print(cos_sim[4,120])   
print(cos_sim[3,24])

0.17227613110014967
-0.22706217338271198


In [65]:
# Obtener candidatos para una pista dada

# Dada una pista del user-input (pos = 0, 1, ..., 6) extraer "ncands" candidatos, usando
# "cos_sim" y siempre y cuando superen un umbral de similitud

def obtener_candidatos(pos, cos_sim, ncands, umbral = 0.8):
    # Obtener todas las pistas candidatas por encima de umbral
    idx = np.where(cos_sim[pos,:]>=umbral)[0] # ejm. idx: [27, 82, 135]
    
    # Y organizarlas de forma descendente (por similitudes de mayor a menor)
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] porque por defecto argsort organiza de manera ascendente

    # Si hay más de "ncands", retornar únicamente un total de "ncands"
    if len(idx) >= ncands:
        cands = idx[0:ncands]
    else:
        cands = idx
  
    return cands

In [66]:
# Ejemplo de uso
# Todo: Cuantas canciones necesitará el usuario
# para que el sistema le recomiende una canción
# que le guste?

for i in range(userinput_df.shape[0]):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i} ==> pistas candidatas: {cands}, similitudes: {cos_sim[i,cands]}')

0 ==> pistas candidatas: [], similitudes: []
1 ==> pistas candidatas: [ 51   2 320  69 513], similitudes: [0.91655146 0.91134643 0.90987351 0.90630753 0.88987016]
2 ==> pistas candidatas: [], similitudes: []
3 ==> pistas candidatas: [320], similitudes: [0.8665947]
4 ==> pistas candidatas: [], similitudes: []
5 ==> pistas candidatas: [124 459 242 379 160], similitudes: [0.93412313 0.85858239 0.84843363 0.83010912 0.82975907]
6 ==> pistas candidatas: [ 74  52 188], similitudes: [0.84939719 0.83914368 0.80361963]
7 ==> pistas candidatas: [436 157 526], similitudes: [0.86872412 0.83921593 0.83234102]
8 ==> pistas candidatas: [336 437 244], similitudes: [0.87037572 0.8570547  0.82460865]
9 ==> pistas candidatas: [438 314  98], similitudes: [0.88547769 0.87207664 0.82236977]
10 ==> pistas candidatas: [], similitudes: []
11 ==> pistas candidatas: [], similitudes: []
12 ==> pistas candidatas: [], similitudes: []
13 ==> pistas candidatas: [], similitudes: []
14 ==> pistas candidatas: [202], sim

In [67]:
# Para crear la playlist se requieren únicamente los ids
ids_user = []
ids_playlist = []

for i in range(userinput_df.shape[0]):
    print(userinput_df.index[i])   # Nombre de la pista en el top-20
    ids_user.append(userinput_df['id'][i])
    
    # Obtener listado de candidatos para esta pista
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    
    # Si hay pistas relacionadas obtener los ids correspondientes
    # e imprimir en pantalla
    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        # Obtener los ids correspondientes e imprimir en pantalla
        for j in cands:
            id_cand = candidatos_df['id'][j]
            ids_playlist.append(id_cand)
            
            # E imprimir en pantalla el candidato
            print(f'   {candidatos_df.index[j]}')

La Chata
     ***No se encontraron pistas relacionadas***
Decir adiós
   (I Can't Help) Falling In Love With You - Can't Help Falling In Love
   Entraré
   Acurrucar
   Tonight I Celebrate My Love
   El lugar correcto
Te Quiero
     ***No se encontraron pistas relacionadas***
Decir Adiós
   Acurrucar
Fin del Tiempo
     ***No se encontraron pistas relacionadas***
Un Vino, Una Cerveza
   Las Solteras
   LACONE
   Toda la Vida
   Tu Angelito
   Esa Sí Es una Mujer
Cuéntame
   Laxed – Siren Beat
   M-40
   Ay Amor
Los Globos del Cielo
   Los Globos del Cielo
   Verano Del '57
   Señal De Vida
Me Estoy Enamorando
   La Ciguapa
   Me Estoy Enamorando
   María
Degeneración Actual
   Degeneración Actual
   Berlin
   Auto Rojo
Fiesta Pagana
     ***No se encontraron pistas relacionadas***
Molinos De Viento
     ***No se encontraron pistas relacionadas***
Haste Que El Cuerpo Aguante
     ***No se encontraron pistas relacionadas***
La Costa Del Silencio
     ***No se encontraron pistas relaciona

In [68]:
# Eliminar candidatos que ya están en el top-20
ids_playlist_dep = [x for x in ids_playlist if x not in ids_user]

# Y eliminar posibles repeticiones
ids_playlist_dep = list(set(ids_playlist_dep))
ids_user = list(set(ids_user))

In [69]:
ids_user.extend(ids_playlist_dep)

In [70]:
ids_user

['58t9Q8VqXGhaqtilMkIkRx',
 '1snT64YtuTvZYVzuornRjw',
 '7c6Zu7NeLHWy4r3ztZuT0G',
 '5Q2J37xbIR60z6ifswoGKe',
 '02dphTJYUQ9pmdNC52iyOz',
 '19vhfSUgVJO2enJ6XidUGO',
 '0eJGouVmEGxPktWPSTfpuI',
 '4COvULULVKLsMKMRKIiXUa',
 '0gnsuw6eGZEwph1rKnxzOu',
 '5xj5qixWeH2I44SFx0Icdv',
 '5Q2uU5NGcS0mKwmgF28kRZ',
 '3jzpdDnLaQcsuGCw4JwVYS',
 '0dQPkfweyWDzZ20Auq3F14',
 '76KLnsqCiFEFTEQp2Vaf8R',
 '4mTsRdFE28MYlrZo8N4Es0',
 '1vywengPlpJuJggJ9xOUYB',
 '1koXbetRP1PgmDRqU4DFPb',
 '5PSCWHpXi8I45NXURHyhBA',
 '3BJiF37kmX4HCfYSO7qBnQ',
 '7M8sGCxjrx4tKV1m0g9JDA',
 '6Fu5E5nuKfVOecwBmWLi7O',
 '3G6XpCfczuhbTSBlgfqe9v',
 '2IzV2SPZQLadtA46ni3ICq',
 '0XXbz44sfkVbDgs2c7sKOS',
 '4Rjq1TZbF0bDstOs6SiI6J',
 '3WXURJDWa3YGUQtVMYrXfK',
 '2mTvV8CHQ44qaXxaD2FUkP',
 '77VXEopCatM9pqJl0beeCj',
 '4BXG52ul7GlV4Qpve6JDJo',
 '4JPKPMBj03iHQ6zrOJzEKd',
 '6kFJXL8yuKMxq6oVLbjGgO',
 '1xYBeQ5u8uaJ3oBWktjGyJ',
 '7JOyY9GA3P2Evx50oetDKb',
 '2WUV07tqCF5JR3bEjpYHVg',
 '2ckJCt5gFJ1tB4etHN90TD']

## Crear la playlist en spotipy

In [75]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [76]:
# Crear la playlist en spotify
me = sp.me()
username = me['id']

pl = sp.user_playlist_create(user=username, 
                             name='DJ Spotify',
                             description='Playlist creada por el DJ Spotify',
                             public=False)

In [77]:
sp.playlist_add_items(pl['id'], ids_user)

{'snapshot_id': 'MywxYzE4OGUwNGY2NDA1MmNjZmI0ZDY3MjgxNjgxMWIwOTI4Y2E3ODYx'}